In [14]:
# importing dependencies
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os

# import openweather API key
from Config import g_key

In [15]:
#importing the csv from part1
path = "../Resources/CityWeather.csv"
Vacation_DF = pd.read_csv(path, low_memory = False)

In [16]:
#putting api keys and locations&humidity
gmaps.configure(api_key=g_key)
locations = Vacation_DF[["Latitude","Longitude"]].astype(float)
humidity = Vacation_DF["Humidity"].astype(float)

In [17]:
# heatmaps for google
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# now that the heatmap is working, we need to narrow down the dataframe for MY ideal weather condition for vacation.

Hotel_df = Vacation_DF.loc[ (Vacation_DF["Max Temp"]<77) & (Vacation_DF["Max Temp"]>=70) & (Vacation_DF["Wind Speed"]<10)
                           & (Vacation_DF["Cloudiness"]==0)]


In [19]:
#dropping unnamed column
Hotel_df.drop(["Unnamed: 0"], axis = 1, inplace = True)

#creating an empty column for the hotel name
Hotel_df["Hotel Name"] = ""

C:\Users\yongs\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\yongs\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
#setting up google parameters:
parameters = {"radius":5000, "types":"hotel", "keyword":"hotel", "key":g_key}

In [21]:
# using locations to get the nearest hotels

for index, x in Hotel_df.iterrows():
    lat = x["Latitude"]
    long = x["Longitude"]
    
    parameters["location"] = f"{lat},{long}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(url, params=parameters).json()
    
    try:
        Hotel_df.loc[index, "Hotel Name"] = hotel_request["results"][0]["name"]
    except (KeyError, IndexError):
        print("No Result. Skipped!!")



C:\Users\yongs\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No Result. Skipped!!
No Result. Skipped!!
No Result. Skipped!!


In [26]:
Hotel_df.head(50)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,Puerto Suárez,-18.95,-57.80,72.25,38,0,3.42,BO,1597118828,Hotel casa real
9,Urubichá,-15.38,-62.95,74.50,50,0,2.80,BO,1597118834,
136,Charters Towers,-20.10,146.27,76.91,28,0,6.06,AU,1597118944,Commercial Hotel
152,Lixouri,38.20,20.43,73.40,78,0,2.44,GR,1597118959,Kefalonia Grand Hotel
162,Strezhevoy,60.73,77.59,75.20,60,0,8.95,RU,1597118932,"Гостиница ""Турист"""
168,Tataouine,32.93,10.45,73.40,78,0,3.36,TN,1597118973,Sangho Privilege Tataouine
216,Guerrero Negro,27.98,-114.06,71.76,67,0,8.72,MX,1597118832,Hotel Los Caracoles
228,Glamoč,44.05,16.85,73.99,54,0,1.07,BA,1597119025,Hotel Split
229,Ilhabela,-23.78,-45.36,70.68,60,0,4.74,BR,1597119026,Ilha Flat Hotel
325,Turukhansk,65.82,87.98,73.83,49,0,2.26,RU,1597119105,"ООО ""Туруханская пушнина"""


In [25]:
Hotel_df.dropna(how='any', inplace = True)

C:\Users\yongs\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
